In [3]:
import json
import torch
import custom_utils
import numpy as np
from sentence_transformers import SentenceTransformer
bert = SentenceTransformer('all-MiniLM-L6-v2')

/home/xuxa/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
from nltk.tokenize import word_tokenize
from collections import defaultdict

def tokenize(texts):
    """Tokenize texts, build vocabulary and find maximum sentence length.
    
    Args:
        texts (List[str]): List of text data
    
    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """

    max_len = 0
    tokenized_texts = []
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building our vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        tokenized_sent = word_tokenize(sent)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))

    return tokenized_texts, word2idx, max_len

def encode(tokenized_texts, word2idx, max_len):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)
    
    return np.array(input_ids)

In [5]:
nltk.download('punkt')

NameError: name 'nltk' is not defined

In [12]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_texts, word2idx, max_len = tokenize(X)
input_ids = encode(tokenized_texts, word2idx, max_len)


Tokenizing...



In [13]:
word2idx

{'<pad>': 0,
 '<unk>': 1,
 'ME': 2,
 ':': 3,
 '<': 4,
 'vocalsound': 5,
 '>': 6,
 'Okay': 7,
 '.': 8,
 'Oh': 9,
 'I': 10,
 'totally': 11,
 'disfmarker': 12,
 'Yeah': 13,
 "'cause": 14,
 'moved': 15,
 'it': 16,
 "'S": 17,
 'put': 18,
 'over': 19,
 'here': 20,
 'Then': 21,
 'we': 22,
 'do': 23,
 "n't": 24,
 'have': 25,
 'to': 26,
 'worry': 27,
 'about': 28,
 'UI': 29,
 'Ready': 30,
 'for': 31,
 'this': 32,
 '?': 33,
 'PM': 34,
 'All': 35,
 'set': 36,
 'Cool': 37,
 'Alright': 38,
 ',': 39,
 'is': 40,
 'PowerPoint': 41,
 'time': 42,
 "'ve": 43,
 'done': 44,
 'more': 45,
 'PowerPoints': 46,
 'in': 47,
 'particular': 48,
 'experiment': 49,
 'than': 50,
 'ever': 51,
 'my': 52,
 'life': 53,
 'before': 54,
 'ID': 55,
 'which': 56,
 'kind': 57,
 'of': 58,
 'fun': 59,
 'man': 60,
 'So': 61,
 'uh': 62,
 'our': 63,
 'detailed': 64,
 'design': 65,
 'meeting': 66,
 'where': 67,
 'will': 68,
 'um': 69,
 'look': 70,
 'at': 71,
 'the': 72,
 'prototype': 73,
 'and': 74,
 'right': 75,
 'so': 76,
 'finally

In [14]:
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler,
                              SequentialSampler)

def data_loader(train_inputs, val_inputs, train_labels, val_labels,
                batch_size=50):
    """Convert train and validation sets to torch.Tensors and load them to
    DataLoader.
    """

    # Convert data type to torch.Tensor
    train_inputs, val_inputs, train_labels, val_labels =\
    tuple(torch.tensor(data) for data in
          [train_inputs, val_inputs, train_labels, val_labels])

    # Specify batch_size
    batch_size = 50

    # Create DataLoader for training data
    train_data = TensorDataset(train_inputs, train_labels)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    # Create DataLoader for validation data
    val_data = TensorDataset(val_inputs, val_labels)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

    return train_dataloader, val_dataloader

In [15]:
labels = y

In [16]:
from sklearn.model_selection import train_test_split

# Train Test Split
train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, labels, test_size=0.1, random_state=42)

# Load data to PyTorch DataLoader
train_dataloader, val_dataloader = \
data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=50)

/tmp/ipykernel_143122/3687013642.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tuple(torch.tensor(data) for data in


In [17]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_NLP(nn.Module):
    """An 1D Convulational Neural Network for Sentence Classification."""
    def __init__(self,
                 pretrained_embedding=None,
                 freeze_embedding=False,
                 vocab_size=None,
                 embed_dim=300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):
        """
        The constructor for CNN_NLP class.

        Args:
            pretrained_embedding (torch.Tensor): Pretrained embeddings with
                shape (vocab_size, embed_dim)
            freeze_embedding (bool): Set to False to fine-tune pretraiend
                vectors. Default: False
            vocab_size (int): Need to be specified when not pretrained word
                embeddings are not used.
            embed_dim (int): Dimension of word vectors. Need to be specified
                when pretrained word embeddings are not used. Default: 300
            filter_sizes (List[int]): List of filter sizes. Default: [3, 4, 5]
            num_filters (List[int]): List of number of filters, has the same
                length as `filter_sizes`. Default: [100, 100, 100]
            n_classes (int): Number of classes. Default: 2
            dropout (float): Dropout rate. Default: 0.5
        """

        super(CNN_NLP, self).__init__()
        # Embedding layer
        if pretrained_embedding is not None:
            self.vocab_size, self.embed_dim = pretrained_embedding.shape
            self.embedding = nn.Embedding.from_pretrained(pretrained_embedding,
                                                          freeze=freeze_embedding)
        else:
            self.embed_dim = embed_dim
            self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                          embedding_dim=self.embed_dim,
                                          padding_idx=0,
                                          max_norm=5.0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        # Compute logits. Output shape: (b, n_classes)
        logits = self.fc(self.dropout(x_fc))

        return logits

In [23]:
import torch.optim as optim

def initialize_model(pretrained_embedding=None,
                    freeze_embedding=False,
                    vocab_size=None,
                    embed_dim=300,
                    filter_sizes=[3, 4, 5],
                    num_filters=[100, 100, 100],
                    num_classes=2,
                    dropout=0.5,
                    learning_rate=0.01):
    """Instantiate a CNN model and an optimizer."""

    assert (len(filter_sizes) == len(num_filters)), "filter_sizes and \
    num_filters need to be of the same length."

    # Instantiate CNN model
    cnn_model = CNN_NLP(pretrained_embedding=pretrained_embedding,
                        freeze_embedding=freeze_embedding,
                        vocab_size=vocab_size,
                        embed_dim=embed_dim,
                        filter_sizes=filter_sizes,
                        num_filters=num_filters,
                        num_classes=2,
                        dropout=0.5)
    
    # Send model to `device` (GPU/CPU)
    cnn_model.to(device)

    # Instantiate Adadelta optimizer
    optimizer = optim.Adadelta(cnn_model.parameters(),
                               lr=learning_rate,
                               rho=0.95)

    return cnn_model, optimizer

In [19]:
from sklearn.metrics import f1_score

In [20]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility."""

    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, optimizer, train_dataloader, val_dataloader=None, epochs=10):
    """Train the CNN model."""
    
    # Tracking best validation accuracy
    best_accuracy = 0

    # Start training loop
    print("Start training...\n")
    print(f"{'Epoch'} | {'Train Loss'} | {'Val Loss'} | {'Val Acc'} | {'F1-score'}")
    print("-"*60)

    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================

        # Tracking time and loss
        t0_epoch = time.time()
        total_loss = 0

        # Put the model into the training mode
        model.train()

        for step, batch in enumerate(train_dataloader):
            # Load batch to GPU
            b_input_ids, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Update parameters
            optimizer.step()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        # =======================================
        #               Evaluation
        # =======================================
        if val_dataloader is not None:
            # After the completion of each training epoch, measure the model's
            # performance on our validation set.
            val_loss, val_accuracy, f1_value = evaluate(model, val_dataloader)

            # Track the best accuracy
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            print(f"{epoch_i + 1} | {avg_train_loss} | {val_loss} | {val_accuracy} | {f1_value}")
            
    print("\n")
    print(f"Training complete! Best accuracy: {best_accuracy:.2f}%.")

def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's
    performance on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled
    # during the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []
    f1_value = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)
        
        preds_np = preds.cpu().numpy()
        b_labels_np = b_labels.cpu().numpy()
        f1_value.append(f1_score(b_labels_np, preds_np))
    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)
    f1_value = np.mean(f1_value)

    return val_loss, val_accuracy, f1_value

In [48]:
URL = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip"
FILE = "fastText"

if os.path.isdir(FILE):
    print("fastText exists.")
else:
    !wget -P $FILE $URL
    !unzip $FILE/crawl-300d-2M.vec.zip -d $FILE

--2023-12-02 10:08:08--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 99.86.91.38, 99.86.91.123, 99.86.91.116, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|99.86.91.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1,4G) [application/zip]
Saving to: ‘fastText/crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1,42G  8,86MB/s    in 2m 3s   

2023-12-02 10:10:11 (11,9 MB/s) - ‘fastText/crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

Archive:  fastText/crawl-300d-2M.vec.zip
  inflating: fastText/crawl-300d-2M.vec  


In [24]:
from tqdm import tqdm_notebook

def load_pretrained_vectors(word2idx, fname):
    """Load pretrained vectors and create embedding layers.
    
    Args:
        word2idx (Dict): Vocabulary built from the corpus
        fname (str): Path to pretrained vector file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())

    # Initilize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm_notebook(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")

    return embeddings

In [24]:
import optuna

def objective(trial):
    # Define the hyperparameters to be optimized
    embed_dim = trial.suggest_int('embed_dim', 50, 300)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    dropout = trial.suggest_uniform('dropout', 0.1, 0.5)

    # Set seed for reproducibility
    set_seed(42)

    # Initialize the model with suggested hyperparameters
    cnn_rand, optimizer = initialize_model(vocab_size=len(word2idx),
                                           embed_dim=embed_dim,
                                           learning_rate=learning_rate,
                                           dropout=dropout)

    # Train the model
    train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=50)

    # Evaluate the model and return the validation accuracy as the objective value
    val_loss, val_accuracy, f1_value = evaluate(cnn_rand, val_dataloader)
    print(val_loss)
    return val_loss

In [25]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and their corresponding validation accuracy
# print('Best trial:')
# trial = study.best_trial
# print('Validation Accuracy: {:.3f}'.format(trial.value))
# print('Best Hyperparameters:', trial.params)

[I 2023-12-04 20:56:34,958] A new study created in memory with name: no-name-dce5ca9f-2f61-4f03-a3f7-1e673e0e1829
/tmp/ipykernel_143122/3687490577.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/tmp/ipykernel_143122/3687490577.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform('dropout', 0.1, 0.5)


Start training...

Epoch | Train Loss | Val Loss | Val Acc | F1-score
------------------------------------------------------------
1 | 0.5200359354006405 | 0.43928947167037286 | 81.69336143308746 | 0.0
2 | 0.4330288100794003 | 0.41041451182267435 | 81.69336143308746 | 0.0
3 | 0.41311916312493313 | 0.3930017733818864 | 81.69336143308746 | 0.0
4 | 0.3993401284862184 | 0.3808402621174512 | 81.69336143308746 | 0.0
5 | 0.39014101739323465 | 0.3721039127813627 | 81.66596417281349 | 0.0
6 | 0.38362657022448854 | 0.36532236584653593 | 81.93782929399369 | 0.035430589197712484
7 | 0.37878954359287514 | 0.3602353598769397 | 82.06111696522656 | 0.07186719092811379
8 | 0.3741923388127887 | 0.35617724163075015 | 82.28029504741835 | 0.11404243137300095
9 | 0.3716903121327406 | 0.35297195388846203 | 82.54056902002108 | 0.16474203118038735
10 | 0.36844412699934176 | 0.3504187479002835 | 82.66385669125395 | 0.1840135567270942
11 | 0.3663537341998805 | 0.348258155870111 | 82.88303477344574 | 0.2162550460

[I 2023-12-04 21:26:57,518] Trial 0 finished with value: 0.3296433805602871 and parameters: {'embed_dim': 158, 'learning_rate': 0.001144328670337944, 'dropout': 0.14915879947400185}. Best is trial 0 with value: 0.3296433805602871.


0.3296433805602871
Start training...

Epoch | Train Loss | Val Loss | Val Acc | F1-score
------------------------------------------------------------


/tmp/ipykernel_143122/3687490577.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/tmp/ipykernel_143122/3687490577.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform('dropout', 0.1, 0.5)


...

0.3181242602941108
Start training...

Epoch | Train Loss | Val Loss | Val Acc | F1-score
------------------------------------------------------------


/tmp/ipykernel_143122/3687490577.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/tmp/ipykernel_143122/3687490577.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform('dropout', 0.1, 0.5)


1 | 0.42557176403040553 | 0.3690834725148057 | 81.87144362486828 | 0.02951048190774218
2 | 0.37102971166736853 | 0.34894371338903085 | 83.00632244467862 | 0.2655796899509294
3 | 0.35983116319830266 | 0.3412897102024457 | 83.23709167544784 | 0.29970446506790105
4 | 0.35360261800183435 | 0.33729796591278627 | 83.38777660695469 | 0.3660688780209937


[W 2023-12-05 08:28:48,752] Trial 22 failed with parameters: {'embed_dim': 268, 'learning_rate': 0.006426567586391541, 'dropout': 0.4609735941332162} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/xuxa/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_143122/3687490577.py", line 19, in objective
    train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=50)
  File "/tmp/ipykernel_143122/3709412887.py", line 46, in train
    logits = model(b_input_ids)
  File "/home/xuxa/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1518, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/xuxa/.local/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1527, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_143122/1224269538.py", line 78, in forward
    

KeyboardInterrupt: 

In [ ]:
# Print the best hyperparameters and their corresponding validation accuracy
print('Best trial:')
trial = study.best_trial
print('Validation Accuracy: {:.3f}'.format(trial.value))
print('Best Hyperparameters:', trial.params)

In [33]:
# CNN-rand: Word vectors are randomly initialized.
set_seed(42)
cnn_rand, optimizer = initialize_model(vocab_size=len(word2idx),
                                      embed_dim=300,
                                      learning_rate=0.1,
                                      dropout=0.5)
train(cnn_rand, optimizer, train_dataloader, val_dataloader, epochs=50)

Start training...

Epoch | Train Loss | Val Loss | Val Acc | F1-score
------------------------------------------------------------
1 | 0.3589555164087074 | 0.3397639045772487 | 83.05900948366701 | 0.22782265152276604
2 | 0.33430641900394853 | 0.3305757246809463 | 83.7850368809273 | 0.31662686787832695
3 | 0.3213110337438295 | 0.32292095469693616 | 84.19599578503689 | 0.41895430079075435
4 | 0.3134392347152627 | 0.32153395042844013 | 84.71654373024236 | 0.4273816042405493
5 | 0.30223780927968863 | 0.320953333112475 | 84.70284510010538 | 0.4786553577790299
6 | 0.2913446164482048 | 0.3220095121084827 | 84.77133825079031 | 0.480933064966218
7 | 0.2819221992535511 | 0.3240414533304842 | 84.62065331928346 | 0.5093598484627935
8 | 0.27080093700131147 | 0.32725794695011556 | 84.81243414120127 | 0.47227554815107375
9 | 0.26123465762248643 | 0.33292579732529104 | 84.42887249736565 | 0.5030228731537973
10 | 0.2484275270538228 | 0.3363199178894905 | 84.53846153846153 | 0.5255026088724173
11 | 0.23

In [34]:
def predict(text, model=cnn_rand.to("cpu"), max_len=62):
    """Predict probability that a review is positive."""

    # Tokenize, pad and encode text
    tokens = word_tokenize(text.lower())
    padded_tokens = tokens + ['<pad>'] * (max_len - len(tokens))
    input_id = [word2idx.get(token, word2idx['<unk>']) for token in padded_tokens]

    # Convert to PyTorch tensors
    input_id = torch.tensor(input_id).unsqueeze(dim=0)

    # Compute logits
    logits = model.forward(input_id)

    #  Compute probability
    probs = F.softmax(logits, dim=1).squeeze(dim=0)

    print(f"This review is {probs[1] * 100:.2f}% important.")

In [7]:
# read labels
with open("training_labels.json", "r") as json_file:
    labels = json.load(json_file)

# read nodes and edges
nodes, edges = custom_utils.gather_dataset("training")

In [8]:
# aggregate data from all dialogs
X, y, edge_idx = [], [], [] 
count = 0
for id in nodes.keys():
        X += nodes[id]
        y += labels[id]
        edge_idx += [[e[0] + count, e[1] + count] for e in edges[id]]
        count += len(labels[id])

In [9]:
from sklearn.model_selection import StratifiedShuffleSplit

# stratified split (we do it every epoch)
def split(spliter, X, y):
    train_idx, val_idx = list(spliter.split(X, y))[0]
    train_mask, val_mask = [False]*len(X), [False]*len(X)
    for idx in train_idx: train_mask[idx] = True
    for idx in val_idx: val_mask[idx] = True
    train_mask = torch.Tensor(train_mask).bool()
    val_mask = torch.Tensor(val_mask).bool()
    return train_mask, val_mask


sss = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state=42)
train_mask, val_mask = split(sss, X, y)

In [10]:
# formatting
y = torch.Tensor(y).long()
edge_idx = torch.Tensor(edge_idx).long().transpose(0,1)

In [11]:
X

['ME: <vocalsound> Okay .',
 'ME: Oh',
 'ME: I totally <disfmarker>',
 'ME: Yeah',
 "ME: 'cause I moved it . <vocalsound>",
 "ME: 'S put it over here .",
 "ME: Then we don't have to worry about it .",
 'UI: <vocalsound> Ready for this ?',
 'PM: All set ?',
 'PM: Cool .',
 'PM: Alright ,',
 'PM: it is PowerPoint time .',
 "PM: I've done more PowerPoints in this particular experiment than I've ever done in my life before this experiment <vocalsound>",
 'ID: <vocalsound> Yeah <vocalsound> .',
 'PM: which is kind of fun .',
 'UI: <vocalsound> Oh man . <vocalsound>',
 'PM: So uh',
 'PM: here we have our detailed design meeting',
 'PM: where we will um look at the prototype',
 'PM: and',
 'PM: um <vocalsound> <disfmarker> right so um ,',
 'PM: I finally figured out what this whole second bullet point is about in my <disfmarker>',
 'PM: that my coach was sending to me .',
 "PM: It means I'm supposed to read the minutes from the previous meeting .",
 'ID: Oh',
 'ID: really ?',
 'ID: Okay .',
 

In [15]:
X = bert.encode(X, show_progress_bar=True, convert_to_tensor=True)

Batches:   0%|          | 0/2270 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [14]:
num_features = X.shape[1]
num_classes = 2

In [15]:
from torch.nn import Linear
import torch.nn.functional as F

class MLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(num_features, 300)
        self.lin2 = Linear(300, 600)
        self.lin3 = Linear(600, num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin3(x)
        return x

In [19]:
def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(X)  # Perform a single forward pass.
      # out = model(X, edge_idx)  # Perform a single forward pass.
      loss = criterion(out[train_mask], y[train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(X)
      # out = model(X, edge_idx)
      pred = out.argmax(dim=1)  # Use the class with highest probability.

      # TP, FP, FN
      TP = ((pred[val_mask] == 1) & (y[val_mask] == 1)).sum()
      FP = ((pred[val_mask] == 1) & (y[val_mask] == 0)).sum()
      FN = ((pred[val_mask] == 0) & (y[val_mask] == 1)).sum()

      # Calculate precision, recall, and F1 score
      precision = TP / max((TP + FP), 1e-10)  # Avoid division by zero
      recall = TP / max((TP + FN), 1e-10)  # Avoid division by zero
      f1_score = 2 * (precision * recall) / max((precision + recall), 1e-10)  # Avoid division by zero

      # Calculate accuracy
      test_correct = pred[val_mask] == y[val_mask]
      test_acc = int(test_correct.sum()) / int(val_mask.sum())

      # return criterion(model(X, edge_idx)[val_mask], y[val_mask]), f1_score
      return criterion(model(X)[val_mask], y[val_mask]), f1_score

In [20]:
for epoch in range(1, 201):
    train_mask, val_mask = split(sss, X, y)
    train_loss = train()
    val_loss, f1_score = test()
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, F1-score: {f1_score:.4f}')

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [72623, 384]

In [21]:
model = MLP()
# model = GCN(hidden_channels=500)
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)  # Define optimizer.

In [22]:
def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(X)  # Perform a single forward pass.
      # out = model(X, edge_idx)  # Perform a single forward pass.
      loss = criterion(out[train_mask], y[train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(X)
      # out = model(X, edge_idx)
      pred = out.argmax(dim=1)  # Use the class with highest probability.

      # TP, FP, FN
      TP = ((pred[val_mask] == 1) & (y[val_mask] == 1)).sum()
      FP = ((pred[val_mask] == 1) & (y[val_mask] == 0)).sum()
      FN = ((pred[val_mask] == 0) & (y[val_mask] == 1)).sum()

      # Calculate precision, recall, and F1 score
      precision = TP / max((TP + FP), 1e-10)  # Avoid division by zero
      recall = TP / max((TP + FN), 1e-10)  # Avoid division by zero
      f1_score = 2 * (precision * recall) / max((precision + recall), 1e-10)  # Avoid division by zero

      # Calculate accuracy
      test_correct = pred[val_mask] == y[val_mask]
      test_acc = int(test_correct.sum()) / int(val_mask.sum())

      # return criterion(model(X, edge_idx)[val_mask], y[val_mask]), f1_score
      return criterion(model(X)[val_mask], y[val_mask]), f1_score

In [23]:
for epoch in range(1, 201):
    train_mask, val_mask = split(sss, X, y)
    train_loss = train()
    val_loss, f1_score = test()
    print(f'Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Val loss: {val_loss:.4f}, F1-score: {f1_score:.4f}')

Epoch: 001, Train Loss: 0.6664, Val loss: 0.4499, F1-score: 0.0000
Epoch: 002, Train Loss: 0.4510, Val loss: 0.6845, F1-score: 0.0000


KeyboardInterrupt: 

In [69]:
test_acc, f1_score = test()
print(f'Test Accuracy: {test_acc:.4f}, F1-score: {f1_score:.4f}')

Test Accuracy: 0.3364, F1-score: 0.5692


In [70]:
test_nodes, test_edges = custom_utils.gather_dataset("test")

test_labels = {}
model.eval()
for id, sentences in test_nodes.items():
    X_test = bert.encode(sentences, convert_to_tensor=True)
    out = model(X_test)
    pred = out.argmax(dim=1)
    test_labels[id] = pred.tolist()

In [71]:
# # uncomment to generate test labels
# with open("test_labels.json", "w") as json_file:
#     json.dump(test_labels, json_file)